# Introduction to Sionna RT - Assets & BSDFs Configuration Tools

In this notebook, you will
- Learn how to programaticaly append or remove assets object to a scene (without using Blender).
- See the impact of the assets within the scene w.r.t. ray-traced channels for link-level simulations instead of stochastic channel models.
- Learn how to define, set and change the rendering properties of a radio material in the scene using BSDF object.


## Table of Contents
* [Information On Assets](#Information-On-Assets)
*

## Information On Assets

It is usefull to be able to construct a scene dynamically by adding and removing objects. To this end, Sionna incorporates a few functionalities. 

One can define a scene, e.g. using Blender, import the scene within Sionna and then add new assets (object or group of objects) to that scene. The assets can be separately defined in Blender. 

As an example, one could define a scene with a car park, and append to that scene a varing number of cars to see the impact in terms of RF propagation. Using this mechanism, the user should be able to automatically generate datasets from the scenes and/or define complex scenario.


## GPU Configuration and Imports <a class="anchor" id="GPU-Configuration-and-Imports"></a>

In [ ]:
import os
gpu_num = 0 # Use "" to use the CPU
os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Colab does currently not support the latest version of ipython.
# Thus, the preview does not work in Colab. However, whenever possible we
# strongly recommend to use the scene preview mode.
try: # detect if the notebook runs in Colab
    import google.colab
    colab_compat = True # deactivate preview
except:
    colab_compat = False
resolution = [480,320] # increase for higher quality of renderings

# Allows to exit cell execution in Jupyter
class ExitCell(Exception):
    def _render_traceback_(self):
        pass

# Import Sionna
try:
    import sionna
except ImportError as e:
    # Install Sionna if package is not already installed
    import os
    os.system("pip install sionna")

# Configure the notebook to use only a single GPU and allocate only as much memory as needed
# For more details, see https://www.tensorflow.org/guide/gpu
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)
# Avoid warnings from TensorFlow
tf.get_logger().setLevel('ERROR')

tf.random.set_seed(1) # Set global random seed for reproducibility


# To Remove

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

# Import Sionna RT components
from sionna.rt import load_scene, Transmitter, Receiver, PlanarArray, AssetObject, BSDF, RadioMaterial

# For link-level simulations
from sionna.channel import cir_to_time_channel

## Loading Scenes

The Sionna RT module can either load external scene files (in Mitsuba's XML file format) or it can load one of the [integrated scenes](https://nvlabs.github.io/sionna/api/rt.html#example-scenes).

In this example, we load a scene containing a floor and a basic wall above it. The scene can be seen as a the static reference, in which one can add or remove assets. 

In [ ]:
# Load scene
scene = load_scene(filename=sionna.rt.scene.floor_wall)


if colab_compat:
    scene.render(camera="scene-cam-0", num_samples=512);
    raise ExitCell
scene.preview()

## Load and Manage Assets

In a static scene, one can add assets at various position to evaluate different scenario without having to design a dedicated scene beforehand. It should be noted, that assets are added to the scene by recreating the scene from scratch (reloading the scene). Hence, the action itself to add or remove an asset object to a scene is not differentiable, but the previous and the novel scenes preserve their differentiability properties as any scene in Sionna. This tool must be seen as a way to programmatically construct the scene, e.g. to generate many random scenes to construct a dataset for neural networks training.

The actions that trigger a scene reload will be presented throughout this notebook, as well as the impact of reloading the scene.

Some preloaded assets are located in sionna/rt/assets. Let's see how to load and use them.

NB: As a workaround to maintain differentiability when placing an object, one could use the mobility tools from Sionna to move pre-existing objects of the scene into or out of the zone of interest of the scene. Yet, this would require to have put these objects in the scene in the first place.

In [ ]:
# Load an asset
body_asset = AssetObject(name="asset_0", filename=sionna.rt.asset_object.body)

# Alternative call
# body_asset = AssetObject(name="asset_0", filename="./sionna/rt/assets/body/body.xml")

# Add the asset to the scene. Adding an AssetObject to the scene trigger a scene reload.
print(f"Memory address of the Scene object: {hex(id(scene))}")
scene.add(body_asset)
print(f"Memory address of the Scene object: {hex(id(scene))}")

# Preview the scene
if colab_compat:
    scene.render(camera="scene-cam-0", num_samples=512)
    raise ExitCell
scene.preview()

As it can be seen in the preview, a body has been added to the previous scene. This action triggered a scene reload.

Reloading the scene does not create a new Scene object.

The body can be modified in the same way as any object of the scene.

It is listed in the .asset_objects method from the scene.

In [ ]:
# Print a dictionnary containing the assets of the scene
print(scene.asset_objects)

The dictionnary contains all the assets of the scene.

Our body "asset_0" is listed as an AssetObject. They are slightly different from others Sionna's SceneObject, as we will demonstrate in this notebook. 

First of all, AssetObject can be composed of several SceneObjects. Use the method .shapes to display them.

In [ ]:
# Get the asset object by name as any object from the scene
body_asset = scene.get('asset_0')
print(f"Check the type of the body_asset variable: {type(body_asset).__name__}")

# Print the objects (SceneObject) composing the asset (AssetObject)
print(f"{body_asset.shapes}")

Our asset (AssetObject) "asset_0" is composed of one object (SceneObject) "asset_0_body".

The name of the object is based on the asset name "asset_0" associated with the component name defined in Blender "body".

The method AssetObject.shapes return a dictionnary, containing all the SceneObject that composed the asset.

You should have noted that the dictionnary contains a so called weakproxy so the SceneObject. You can handle this object as any SceneObject since it is basically a proxy to the SceneObject. 

The only major difference is that the weakproxy is a weak reference to the SceneObject. The only strong reference to the SceneObject is contained in the Scene itself. Getting the SceneObject from the scene or the assets should only return a weak reference. The reason behind this design choice is explained later, when we will remove an AssetObject from the scene.

To learn more on weak reference, see [here](https://docs.python.org/3.12/library/weakref.html).

In [ ]:
# Print a dictionnary containing all the objects of the scene
print(scene.objects) # also return weak references

The scene contains three objects:
- The "floor" that was in the basic scene.
- The "wall" that was also in the basic scene.
- The "body", named "asset_0_body", that we have added.

Hence, the body is seen as a classic SceneObject by Sionna.

In [ ]:
# Get the body object from the scene as any SceneObject
body_object = scene.get('asset_0_body') # also return a weak reference

print(f"Current body asset position: {body_asset.position.numpy()}")
print(f"Current body asset orientation: {body_asset.orientation.numpy()}")
print(f"Current body asset velocity: {body_asset.velocity.numpy()}")

print(f"Current body object position: {body_object.position.numpy()}")
print(f"Current body object orientation: {body_object.orientation.numpy()}")
print(f"Current body object velocity: {body_object.velocity.numpy()}")

The asset also exposes the usual attributes "position", "orientation" and "velocity".

As it can be seen, the position of the object is not [0.,0.,0.], contrary to the asset one. This is because Sionna creates a virtual rectangular Axis-Aligned Bounding Box (AABB) around the whole asset and return the position of the barycenter of this box. Hence, the position is somewhere at the center of the body. The asset position, on the contrary, is the origin defined originally in Blender; here at the feet of the body. To witness that behavior, we can simply fix the position of the object to [0.,0.,0.] using the SceneObject property.

In [ ]:
# Store the original position
original_body_object_position = body_object.position.numpy()

# Manually set the SceneObject position
body_object.position = [0.,0.,0.]

print(f"Current body object position: {body_object.position.numpy()}")
print(f"Current body asset position: {body_asset.position.numpy()}")

# Preview the scene
if colab_compat:
    scene.render(camera="scene-cam-0", num_samples=512)
    raise ExitCell
scene.preview()

The AABB (hit box) barycenter of our asset is now at the coordinates [0.,0.,0.]. The position (and other attributes like orientation and velocity) of the asset is not modified when modifying a component object.

In [ ]:
# Manually set the orientation of the SceneObject
body_object.orientation = [np.pi/4.,np.pi/4.,np.pi/4.]

print(f"Orientation of the object: {body_object.orientation.numpy()}")
print(f"Position of the object: {body_object.position.numpy()}")

# Preview the scene
if colab_compat:
    scene.render(camera="scene-cam-0", num_samples=512)
    raise ExitCell
scene.preview()

After the change of orientation of the SceneObject, the position of the barycenter of the object has changed. That is because Sionna's AABB is not rotated, but recomputed with edges parallel to the coordinate reference axis (x,y,z), hence modifying the barycenter position.

That's why we advice to directly manipulate the asset itself, as you will be able to conveniently and consistently manipulate group of objects together. Also, this is more natural, as the origin of the asset is defined by the user through the asset generation in Blender.

This is also the reason why modifying the position/orientation/velocity of an object from an asset will not modify the asset's configuration.

In [ ]:
# Show that the position and orientation of the asset has not been modified
print(f"Position of the asset: {body_asset.position.numpy()}")
print(f"Orientation of the asset: {body_asset.orientation.numpy()}")

# Position the body at its original position, with original orientation
# First set the orientation and then the position to avoid side effect
body_object.orientation = [0.,0.,0.]
body_object.position = original_body_object_position

print(f"Position of the object: {body_object.position.numpy()}")
print(f"Orientation of the object: {body_object.orientation.numpy()}")

# Preview the scene
if colab_compat:
    scene.render(camera="scene-cam-0", num_samples=512)
    raise ExitCell
scene.preview()

The body is back on its feet! Now let's move it to one side of the wall using the asset properties.

In [ ]:
# Move the body to one side of the wall
body_asset.position = [1.,0.,0.]

print(f"Position of the asset: {body_asset.position.numpy()}")
print(f"Orientation of the asset: {body_asset.orientation.numpy()}")

print(f"Position of the object: {body_object.position.numpy()}")
print(f"Orientation of the object: {body_object.orientation.numpy()}")

# Preview the scene
if colab_compat:
    scene.render(camera="scene-cam-0", num_samples=512)
    raise ExitCell
scene.preview()

The body object position has been correctly moved along with the asset one. Now let's change its orientation so it faces the wall.

In [ ]:
# Make the body face the wall
body_asset.orientation = [-np.pi/2.,0.,0.]

print(f"Position of the asset: {body_asset.position.numpy()}")
print(f"Orientation of the asset: {body_asset.orientation.numpy()}")

print(f"Position of the object: {body_object.position.numpy()}")
print(f"Orientation of the object: {body_object.orientation.numpy()}")

# Preview the scene
if colab_compat:
    scene.render(camera="scene-cam-0", num_samples=512)
    raise ExitCell
scene.preview()

The center of the body asset is located at its feet. So we can turn it upside down.

In [ ]:
# Turn the body upside down
body_asset.orientation += [0.,np.pi,0.]

print(f"Position of the asset: {body_asset.position.numpy()}")
print(f"Orientation of the asset: {body_asset.orientation.numpy()}")

print(f"Position of the object: {body_object.position.numpy()}")
print(f"Orientation of the object: {body_object.orientation.numpy()}")

# Preview the scene
if colab_compat:
    scene.render(camera="scene-cam-0", num_samples=512)
    raise ExitCell
scene.preview()

## Adding a Second Asset

Now, let's add another asset body to the scene.

In [ ]:
# Rename the reference to the first asset and object
first_body_asset = body_asset
first_body_object = body_object

# Print the first body object memory address
print(f"Memory address of the first body (SceneObject): {repr(first_body_object)}")

# Create a second body asset
second_body_asset = AssetObject(name="asset_1", filename=sionna.rt.asset_object.body)

# Add the second asset to the scene
scene.add(second_body_asset) # trigger a scene reload

# Show that the first body object is still the same after the scene relaod
print(f"Memory address of the first body (SceneObject): {repr(first_body_object)}") # Same SceneObject, the weak reference is still alive
# Same behavior by getting the object again from the scene
print(f"Memory address of the first body (SceneObject): {repr(scene.get('asset_0_body'))}") # Same SceneObject, the weak reference is still alive

# Preview the scene
if colab_compat:
    scene.render(camera="scene-cam-0", num_samples=512)
    raise ExitCell
scene.preview()

Again, adding an asset to a scene will reload the scene.

All the objects already present in the scene, that should still be present after the reload, are set back to their previous state.

Here, the "asset_0_body" attributes (position/orientation/velocity) are set back, so that it seems that the body never moved.

The SceneObject (in the python sense) are the same, as it can be seen from the memory address of the SceneObject. Also, the weak references are still alive.

In [ ]:
# Check that the position of the asset_0 asset and asset_0_body object are set correctly
print(f"Position of the asset: {first_body_asset.position.numpy()}")
print(f"Orientation of the asset: {first_body_asset.orientation.numpy()}")
print(f"Position of the object: {first_body_object.position.numpy()}")
print(f"Orientation of the object: {first_body_object.orientation.numpy()}")

Now let's move our second asset, as we have for the first one.

In [ ]:
# Show that the second asset was added to the scene
print("Dictionnary of the assets in the scene.")
print(scene.asset_objects)

# Show that the second body was added to the scene
print("Dictionnary of the objects in the scene.")
print(scene.objects)

# Get a reference to the second body object
second_body_object = scene.get("asset_1_body") # return a weak reference

# Position the second asset/body, so that it mirrors the first one
second_body_asset.position = [1.,0.,0.]
second_body_asset.orientation = [-np.pi/2.,0.,0.]

# Preview the scene
if colab_compat:
    scene.render(camera="scene-cam-0", num_samples=512)
    raise ExitCell
scene.preview()

## Removing an Asset

We can remove an asset from a scene. This action triggers a scene reload.

In [ ]:
# We remove the first asset
scene.remove("asset_0") # trigger scene reload

print(f"Scene assets: {scene.asset_objects}")
print(f"Scene objects: {scene.objects}")

# Preview the scene
if colab_compat:
    scene.render(camera="scene-cam-0", num_samples=512)
    raise ExitCell
scene.preview()

The first asset has correctly been removed from the scene. All the component SceneObject (here the first body) have also been removed.

The first thing to note, is that the AssetObject itself is not deleted outside of the scene.

In [ ]:
# Show that the first AssetObject is still available
print(first_body_asset)

Hence it can be added again to the scene if needed.

The second thing to note, is that the removed SceneObject(s) are no longer referenced. More precisely, the strong reference in the scene has been deleted, removing any access through the weak references.

In [ ]:
# The weak references are dead
try:
    print(first_body_object)
except ReferenceError as e:
    print(f"Trying to access the SceneObject returned the error: {e}")

# The dictionnary of the asset is empty
print(f"The dictionnary of the component objects of the asset is equal to: {first_body_asset.shapes}")

The main objective was to help the user and avoid errors, where one would try to access and modify SceneObjects that are not present in the scene anymore.

The following cells presents the way to access to objects of the scene and some specific behavior.

In [ ]:
# get method directly return a weakproxy reference to the SceneObject
second_body_object_get = scene.get('asset_1_body')
print(f"Show the SceneObject: {second_body_object_get}")
print(f"Show the weakproxy and the SceneObject: {repr(second_body_object_get)}")
print(f"The type is ProxyType: {type(second_body_object_get)}")
print(f"The basic memory address is the proxy one: {hex(id(second_body_object_get))}")

In [ ]:
# shapes attribute from asset return a dictionnary, filled with weakproxy to the component SceneObjects
second_body_asset_shapes = second_body_asset.shapes
print(f"The dictionnary of the component objects of the asset is equal to: {second_body_asset_shapes}")

# Access to the SceneObject as you will with any dictionnary
second_body_object_shapes = second_body_asset_shapes['asset_1_body']
print(f"Show the SceneObject: {second_body_object_shapes}")
print(f"Show the weakproxy and the SceneObject: {repr(second_body_object_shapes)}")
print(f"The type is ProxyType: {type(second_body_object_shapes)}")
print(f"The basic memory address is the proxy one: {hex(id(second_body_object_shapes))}")

In [ ]:
# objects method from the scene return a custom WeakRefDict object
scene_objects_dictionnary = scene.objects
print(f"The dictionnary of the objects of the scene is equal to: {scene_objects_dictionnary}")
      
# Access to the SceneObject as you will with any dictionnary
second_body_object_objects = scene_objects_dictionnary['asset_1_body']
print(f"Show the SceneObject: {second_body_object_objects}")
print(f"Show the weakproxy and the SceneObject: {repr(second_body_object_objects)}")
print(f"The type is ProxyType: {type(second_body_object_objects)}")
print(f"The basic memory address is the proxy one: {hex(id(second_body_object_objects))}")

Adding an asset with the same name than another asset will replace the previous asset with the new one.

The SceneObjects of the previous asset are removed and replaced with new ones.

Only the name of the asset is important, not the content of the asset.

In [ ]:
# Create an asset with the same name
second_body_asset_bis = AssetObject('asset_1', filename=sionna.rt.asset_object.body)

# Add it to the scene remove the previous one, triggers a scene reload and warns the user
scene.add(second_body_asset_bis)

# All the previous weak references are dead
try:
    print(second_body_object_get)
except ReferenceError as e:
    print(f"Trying to access the SceneObject returned the error: {e}")
try:
    print(second_body_object_shapes)
except ReferenceError as e:
    print(f"Trying to access the SceneObject returned the error: {e}")
try:
    print(second_body_object_objects)
except ReferenceError as e:
    print(f"Trying to access the SceneObject returned the error: {e}")
    
# The original asset object still exist, but not in the scene
print(f"The original asset: {second_body_asset}")
# This is the one in the scene
print(f"The new asset: {second_body_asset_bis}")
print(f"Scene assets: {scene.asset_objects}")
print(f"Scene objects: {scene.objects}")

# Preview the scene
if colab_compat:
    scene.render(camera="scene-cam-0", num_samples=512)
    raise ExitCell
scene.preview()

## Multi Component Objects Asset

So now that we have play a little with a single body, let's use a two bodies asset.

In [ ]:
# Remove the body asset
scene.remove('asset_1') # scene reload

print(f"Scene assets: {scene.asset_objects}")
print(f"Scene objects: {scene.objects}")

In [ ]:
# Load the two bodies asset
two_bodies_asset = AssetObject(name="asset_two_bodies", filename=sionna.rt.asset_object.two_persons)

# Add the asset to the scene
scene.add(two_bodies_asset) # scene reload

print(f"Scene assets: {scene.asset_objects}")
print(f"Scene objects: {scene.objects}")

# Preview the scene
if colab_compat:
    scene.render(camera="scene-cam-0", num_samples=512)
    raise ExitCell
scene.preview()

Our asset is composed of two objects "asset_two_bodies_person_1" and "asset_two_bodies_person_2", as it can be seen in the objects list of the scene.

We can use the asset properties to move the bodies around.

In [ ]:
# Get individual objects
body_1_object = scene.get('asset_two_bodies_person_1')
body_2_object = scene.get('asset_two_bodies_person_2')

# Move the two bodies to one side of the wall
two_bodies_asset.position = [1.,0.,0.]

print(f"Position of the asset: {two_bodies_asset.position.numpy()}")
print(f"Orientation of the asset: {two_bodies_asset.orientation.numpy()}")

print(f"Position of the person 1 object: {body_1_object.position.numpy()}")
print(f"Orientation of the person 1 object: {body_1_object.orientation.numpy()}") 

print(f"Position of the person 2 object: {body_2_object.position.numpy()}")
print(f"Orientation of the person 2 object: {body_2_object.orientation.numpy()}")

# Preview the scene
if colab_compat:
    scene.render(camera="scene-cam-0", num_samples=512)
    raise ExitCell
scene.preview()

As explained before, using the asset properties is convenient to move around group of SceneObjects.

Here, the two bodies are both handled by a single asset.

Also, note that the orientation property of the SceneObject is not related to the actual orientation of the bodies, but represent the current angle compared to the one at init.

The center of position and rotation of the asset is on the ground, between the two bodies.

In [ ]:
# Move the two bodies to each side of the wall
two_bodies_asset.position = [0.,0.,0.]
two_bodies_asset.orientation = [np.pi/2.,0.,0.]

print(f"Position of the asset: {two_bodies_asset.position.numpy()}")
print(f"Orientation of the asset: {two_bodies_asset.orientation.numpy()}")

print(f"Position of the person 1 object: {body_1_object.position.numpy()}")
print(f"Orientation of the person 1 object: {body_1_object.orientation.numpy()}") 

print(f"Position of the person 2 object: {body_2_object.position.numpy()}")
print(f"Orientation of the person 2 object: {body_2_object.orientation.numpy()}")

# Preview the scene
if colab_compat:
    scene.render(camera="scene-cam-0", num_samples=512)
    raise ExitCell
scene.preview()

## Interaction with Paths

Now we will demonstrate that the componennt objects of assets have the same interaction with ray tracing than any other Sionna's SceneObject.

We will add a TX/RX couple at each side of the wall.

The bodies objects will be individually moved (even if not recommended), so that we increase the space between them and can use them as reflectors.

In [ ]:
# Place the bodies
two_bodies_asset.orientation = [0.,0.,0.]
body_1_object.position += [0.,1.5,0.]
body_2_object.position += [0.,-1.5,0.]

# Preview the scene
if colab_compat:
    scene.render(camera="scene-cam-0", num_samples=512)
    raise ExitCell
scene.preview()

In [ ]:
# Configure the transmitter and receiver arrays
# These are isotropic antennas
scene.tx_array = PlanarArray(num_rows=1,
                             num_cols=1,
                             vertical_spacing=0.5,
                             horizontal_spacing=0.5,
                             pattern="iso",
                             polarization="V")

scene.rx_array = scene.tx_array

# Add a transmitter and receiver with equal distance from the center of the scene
# The position is set precisely to get reflexions from the bodies hands
scene.add(Transmitter(name="tx", position=[-3.1,0.,2.1]))
scene.add(Receiver(name="rx", position=[3.1,0.,2.1]))

# Preview the scene
if colab_compat:
    scene.render(camera="scene-cam-0", num_samples=512)
    raise ExitCell
scene.preview()

In [ ]:
# Fix the scene frequency
scene.frequency = 2.4e9

# No LOS, one reflection max, exhaustive path tracing
paths = scene.compute_paths(los=False, reflection=True, max_depth=1, num_samples=1e6, method='exhaustive')

print(f"Amplitude of the paths: {np.abs(paths.a[0,0,0,0,0,1:,0].numpy())}")
print(f"Delay of the paths: {paths.tau[0,0,0,1:].numpy()}")

# Open 3D preview (only works in Jupyter notebook)
if colab_compat:
    scene.render("scene-cam-0", paths=paths);
    raise ExitCell
scene.preview(paths=paths)

The bodies provided in the asset are composed of numerous small reflective surfaces. The positions of TX and RX were chosen to obtain reflections on the hands.

NB: To see more paths, one could activate the scattering.

There are exactly 9 paths in the scene:
* The first path to arrive is reflected on the floor.
* The 8 other paths are reflected on the hands. You can zoom on the preview to see the 8 paths.

In the paths list, we can identify which path reflected on which body by looking at the objects property of the paths.

In [ ]:
# Print the object on which each path reflected
print(f"Object ID on which the corresponding path reflected: {paths.objects[0,0,0,1:]}")

# Show object ID of body 1 and 2
print(f"Floor ID: {scene.get('floor').object_id}")
print(f"First body ID: {body_1_object.object_id}")
print(f"Second body ID: {body_2_object.object_id}")

Because of the symetry of the scene, these paths arrive at (almost) the same time with (almost) the same energy.

In [ ]:
# Get the paths amplitude and delay
a, tau = paths.cir()

# Remove first null LOS path
a = a[...,1:,:]
tau = tau[...,1:]

t = tau[0,0,0,:]/1e-9 # Scale to ns
a_abs = np.abs(a)[0,0,0,0,0,:,0]
a_max = np.max(a_abs)

# Add dummy entry at start/end for nicer figure
t = np.concatenate([(0.,), t, (np.max(t)*1.1,)])
a_abs = np.concatenate([(np.nan,), a_abs, (np.nan,)])

# Plot the CIR
plt.figure()
plt.title("Channel impulse response realization")
plt.stem(t, a_abs)
plt.xlim([0, np.max(t)])
plt.ylim([-2e-6, a_max*1.1])
plt.xlabel(r"$\tau$ [ns]")
plt.ylabel(r"$|a|$");

On the CIR, we can see that the 8 paths reflected by the hands arrived at the same time, around 2.45 ns. Generating the time channel estimate from the CIR should merge the power of these paths.

In [ ]:
# Compute the time channel from the CIR
sampling_freq = 5e9 # 0.2ns resolution
l_min = -6
l_max = 20
h_time = cir_to_time_channel(bandwidth=sampling_freq, a=a, tau=tau, l_min=l_min, l_max=l_max, normalize=True)
t_time = np.arange(l_min, (l_max+1), 1) * (1/sampling_freq)

h_time_abs = np.abs(h_time[0,0,0,0,0,0,:].numpy())

# And plot the CIR
plt.figure()
plt.title("Time Channel from CIR")
plt.stem(t_time, h_time_abs)
plt.xlim([0, np.max(t_time)])
plt.xlabel(r"$\tau$ [ns]")
plt.ylabel(r"$|h_{time}|$");

## Radio Materials and BSDFs

In Sionna the materials of the SceneObjects are defined by a RadioMaterial.

A RadioMaterial defines how the radio waves will interact with the object, thanks to attributes like permittivity, conductivity or scattering coefficient.

Sionna defines some radio materials at initialization. All the radio materials currently defined and available can be exposed by the radio_materials dictionnary from the scene:

In [ ]:
print(scene.radio_materials)

This does not mean that they are all currently used in the scene itself, but they are defined in Sionna.

Each RadioMaterial has a name and can be allocated to a SceneObject. Also, we can check all the objects with a given RadioMaterial.

In [ ]:
# Check the radio material of each object of the scene
for obj in list(scene.objects.values()):
    print(f"SceneObject {obj.name} is composed of {obj.radio_material.name}")
    
# Get the objects using each material if any
# The objects are referenced in the RadioMaterial through their object ID
for rm in list(scene.radio_materials.values()):
    obj_list = []
    for obj_id in rm.using_objects:
        for obj in list(scene.objects.values()):
            if obj.object_id == obj_id:
                obj_list.append(obj.name)
    if len(obj_list) != 0:
        print(f"Material {rm.name} is used by {obj_list}")

In [ ]:
# Modify the RadioMaterial of the first person
body_1_object.radio_material = "itu_brick" # does NOT trigger a reload scene

# Alternative call:
# itu_brick_rm = scene.get('itu_brick') # return a RadioMaterial
# body_1_object.radio_material = itu_brick_rm

print(f"{body_1_object.name} is made of {body_1_object.radio_material.name}")

# Check that the first person is now in the list of itu_brick RM, and not anymore in itu_concrete
for rm in list(scene.radio_materials.values()):
    obj_list = []
    for obj_id in rm.using_objects:
        for obj in list(scene.objects.values()):
            if obj.object_id == obj_id:
                obj_list.append(obj.name)
    if len(obj_list) != 0:
        print(f"Material {rm.name} is used by {obj_list}")
    
# Preview the scene
if colab_compat:
    scene.render(camera="scene-cam-0", num_samples=512)
    raise ExitCell
scene.preview()

As you can see, the visual aspect of the first body is still in marble and has not been updated. Changing the radio material of an object do not change the way it is rendered in the scene.

This is because the scene is NOT reloaded after a RadioMaterial assignement, since it would break differentiability. 

To update the rendering, one can manually trigger a scene reload.

It should be noted that RadioMaterial is correctly set and would affect the paths as expected.

In [ ]:
# Check the impact on the paths
paths = scene.compute_paths(los=False, reflection=True, max_depth=1, num_samples=1e6, method='exhaustive')

print(f"Amplitude of the paths: {np.abs(paths.a[0,0,0,0,0,1:,0].numpy())}")
print(f"Delay of the paths: {paths.tau[0,0,0,1:].numpy()}")

# Print the object on which each path reflected
print(f"Object ID on which the corresponding path reflected: {paths.objects[0,0,0,1:]}")

# Show object ID of body 1 and 2
print(f"Floor ID: {scene.get('floor').object_id}")
print(f"First body ID: {body_1_object.object_id}")
print(f"Second body ID: {body_2_object.object_id}")

The paths with amplitude 4.7e-4 bounced on the brick person (first body), and the ones with amplitude 5.7e-4 bounced on the marble person (second body).

In [ ]:
# Manually reload the scene to change the renderring
scene.reload()

# Preview the scene
if colab_compat:
    scene.render(camera="scene-cam-0", num_samples=512)
    raise ExitCell
scene.preview()

After the reload of the scene, the positions of the objects are maintained after the reload. The first body is correctly displayed as a brick object.

The RadioMaterial class does not defined itself how it should be rendered. This is handled by the BSDF class.

A single BSDF is affected to each RadioMaterial defined in Sionna.

At initialization, the BSDF is defined as random. It is said to be "placeholder". Hence one can use any pre-defined RadioMaterial without specifying the BSDF, but we advice to define a proper custom BSDF before that.

The definition of the BSDF can be done manually, or directly through the .xml file of the scene or an asset.

In [ ]:
# Get a RadioMaterial defined in the scene
rm_itu_metal = scene.get("itu_metal")

# Show the BSDF object associated to it
print(f"The BSDF object describing the itu_metal rendering: {rm_itu_metal.bsdf}")
print(f"The BSDF object has a name: {rm_itu_metal.bsdf.name}")
print(f"The corresponding (random) RGB triplet: {rm_itu_metal.bsdf.rgb}")

# Set the second body radio material to itu_metal
body_2_object.radio_material = "itu_metal"

# Manually reload the scene to change the renderring
scene.reload()

# Preview the scene
if colab_compat:
    scene.render(camera="scene-cam-0", num_samples=512)
    raise ExitCell
scene.preview()

We can modify the BSDF manually and set a custom RGB triplet, or equivalently the color using a color name.

The list of available colors can be found [here](https://matplotlib.org/stable/gallery/color/named_colors.html#css-colors)

Modifying the BSDF triggers a scene reload, to modify the rendering.

In [ ]:
# Check if the BSDF is placeholder
print(f"At initialization, the BSDF is a placeholder: {rm_itu_metal.bsdf.is_placeholder}")

# Setting the RGB of a BSDF triggers a reload scene to update the rendering
rm_itu_metal.bsdf.color = 'chartreuse'
print(f"The corresponding chartreuse RGB triplet: {rm_itu_metal.bsdf.rgb}")

# Check if BSDF is placeholder
print(f"After setting the color, the BSDF is not a placeholder anymore: {rm_itu_metal.bsdf.is_placeholder}")

# Preview the scene
if colab_compat:
    scene.render(camera="scene-cam-0", num_samples=512)
    raise ExitCell
scene.preview()

Once the BSDF of a RadioMaterial has been set, it is not a placeholder anymore. It means that the BSDF is seen has set: the user has defined how the material should be rendered.

Placeholder or not, the BSDF can still be modified manually by the user.

In [ ]:
# Create new BSDF object manually
correct_metal_bsdf = BSDF(name='metal_color', color=[69./255.,58/255.,60/255.])
correct_wood_bsdf = BSDF(name='wood_color', color='sienna')

# Get the 'itu_wood' RadioMaterial, it has a placeholder BSDF
rm_itu_wood = scene.get('itu_wood')

# Assign the input BSDF properties to the RadioMaterial BSDFs
rm_itu_metal.bsdf.assign(correct_metal_bsdf) # triggers scene reload
rm_itu_wood.bsdf.assign(correct_wood_bsdf) # triggers scene reload

# Show that the name of the RadioMaterial's BSDF didn't changed
print(f"The name of the itu_wood BSDF is: {rm_itu_wood.bsdf.name}")
print(f"The name of the itu_metal BSDF is: {rm_itu_metal.bsdf.name}")

# Show that the BSDF ot itu_wood is no longer placeholder
print(f"The BSDF of itu_wood is not a placeholder anymore: {rm_itu_wood.bsdf.is_placeholder}")

# Preview the scene
if colab_compat:
    scene.render(camera="scene-cam-0", num_samples=512)
    raise ExitCell
scene.preview()

When you load a scene or add an asset, each shape, a.k.a SceneObject, is associated with a BSDF described in the .xml file of the scene/asset. Then, the RadioMaterial of the SceneObject is infered, based on the name of the BSDF.

Two basic behaviors can be triggered:
* The BSDF / RadioMaterial name is not currently known in Sionna. A new RadioMaterial is added to the scene with the same name and associated with the corresponding BSDF. You still have to specify the RadioMaterial properties for interaction with radio waves.
* The BSDF / RadioMaterial name already exist. The shapes will be associated with the pre-existing RadioMaterial. Regarding the BSDF of the RadioMaterial, if it is still a placeholder, then it will be automatically replaced by the one defined in the .xml file. Otherwise, the standard behavior is to keep the BSDF already specified in the RadioMaterial, and to discard the new BSDF. This is what happened in this notebook, when we loaded the scene itu_brick and itu_concrete BSDFs where overwrited, as well as itu_marble when we loaded our assets.

It is possible to provide a RadioMaterial, by name or object reference, to an asset. This will set the RadioMaterial of all component SceneObject to the given RadioMaterial.

Equivalently, if all the component objects of an asset have the same RadioMaterial, it will automatically set the asset RadioMaterial property.

In [ ]:
print(f"Currently, body 1 is in brick and body 2 is in metal. The asset RadioMaterial is undefined: {two_bodies_asset.radio_material}")

# Set body 1 to the same RadioMaterial as body 2
body_1_object.radio_material = body_2_object.radio_material

print(f"Now that both bodies are in itu_metal, the asset also took that property: {two_bodies_asset.radio_material.name}")

In [ ]:
# Create a new two persons asset and force the RadioMaterial of both bodies as itu_wood
two_bodies_asset_bis = AssetObject(name='asset_2', filename=sionna.rt.asset_object.two_persons, orientation=[-np.pi/2.,0.,0.], radio_material='itu_wood')

# Add it to the scene
scene.add(two_bodies_asset_bis) # triggers scene reload

# Preview the scene
if colab_compat:
    scene.render(camera="scene-cam-0", num_samples=512)
    raise ExitCell
scene.preview()

Changing the RadioMaterial of the asset will change the component SceneObjects ones.

In [ ]:
# Set the RadioMaterial of the two bodies
two_bodies_asset_bis.radio_material = 'itu_marble'

# Check that it worked
print(f"Current RadioMaterial of the asset is: {two_bodies_asset_bis.radio_material.name}")

# Manually reload the scene to update the rendering
scene.reload()

# Preview the scene
if colab_compat:
    scene.render(camera="scene-cam-0", num_samples=512)
    raise ExitCell
scene.preview()

The itu_marble BSDF was defined in the .xml file of the assets body and two_persons.

It means that the BSDF is not placeholder.

Since the BSDF has been defined through an .xml file, it also means that it does not have a RGB/color property.

Setting the RGB/color manually will overwrite the .xml definition of the BSDF.

In [ ]:
# Change manually the BSDF of itu_marble
rm_itu_marble = scene.get('itu_marble')
print(f"Is the BSDF placeholder: {rm_itu_marble.bsdf.is_placeholder}")
print(f"The BSDF does not have a color or RGB: {rm_itu_marble.bsdf.rgb} | {rm_itu_marble.bsdf.color}")
# Manually set the color to overwrite its .xml element
rm_itu_marble.bsdf.color = 'magenta'

# Preview the scene
if colab_compat:
    scene.render(camera="scene-cam-0", num_samples=512)
    raise ExitCell
scene.preview()

In [ ]:
# Clear the scene by removing asset_1
scene.remove('asset_two_bodies') # trigger scene reload

# Create a new asset body. The .xml file describes a itu_marble BSDF
body_asset = AssetObject(name='asset_3', filename=sionna.rt.asset_object.body)

# Add the asset to the scene
scene.add(body_asset) # trigger a scene reload

# Preview the scene
if colab_compat:
    scene.render(camera="scene-cam-0", num_samples=512)
    raise ExitCell
scene.preview()

While a more appropriate color was provided in the asset .xml file for the itu_marble RadioMaterial's BSDF, it has not been selected because the current BSDF is not placeholder.

Yet, this original BSDF can be recovered in the AssetObject.

In [ ]:
# Get the dictionnary of the BSDFs originally present in the asset .xml file
print(f"Dictionnary of original BSDF: {body_asset.original_bsdfs}")

With the original BSDF accessible, the user can manually apply them if necessary.

We can still force the BSDF overwrite by setting the property overwrite_scene_bsdfs to True at the AssetObject creation. It should be noted that the flag takes effect only when the asset is added to the scene.

In [ ]:
# Set the flag to True when creating the AssetObject
body_asset = AssetObject(name='asset_3', filename=sionna.rt.asset_object.body, overwrite_scene_bsdfs=True)

# Add the asset to the scene
scene.add(body_asset) # trigger a scene reload

# Preview the scene
if colab_compat:
    scene.render(camera="scene-cam-0", num_samples=512)
    raise ExitCell
scene.preview()

Since the asset's flag overwrite_scene_bsdfs was set to True before being added to the scene, all the involved BSDFs have been overwritten.

In case you plan to create a new custom RadioMaterial, a random BSDF will be automatically allocated to it.

In [ ]:
# Create a custom RadioMaterial
my_custom_rm = RadioMaterial(name='my_custom_mat')

# Print the random RGB triplet associated
print(f"The BSDF name of my custom RadioMaterial is: {my_custom_rm.bsdf.name}")
print(f"At init, the RGB triplet of my custom RadioMaterial was: {my_custom_rm.bsdf.rgb}")

# Associate my new material with the solo body
body_asset.radio_material = my_custom_rm

# Modify the rendering so it looks like metal
my_custom_rm.bsdf.assign(rm_itu_metal.bsdf) # trigger a scene reload
print(f"The BSDF name of my custom RadioMaterial is still the same: {my_custom_rm.bsdf.name}")
print(f"And the two BSDFs are different python objects: {hex(id(my_custom_rm.bsdf))} is not equal to {hex(id(rm_itu_metal.bsdf))}")

# Alternative way to create the RadioMaterial with the desired BSDF
# my_custom_rm = RadioMaterial(name='my_custom_mat', bsdf=rm_itu_metal.bsdf)

# Preview the scene
if colab_compat:
    scene.render(camera="scene-cam-0", num_samples=512)
    raise ExitCell
scene.preview()

When the custom RadioMaterial is applied to a SceneObject, it is automatically added to the scene.radio_materials dictionnary and to the .xml file describing the scene.

It is not possible to add to the scene a RadioMaterial with the same name as another one. As you may have noticed, the modification of a RadioMaterial property from anywhere (from an AssetObject, a SceneObject, etc) will propagate to all the SceneObject sharing this RadioMaterial. In fact, each RadioMaterial object is unique, so that the scene is coherent.

Yet, there is a mechanism to assign the properties of a RadioMaterial to another.

In [ ]:
# Create antoher cutom RadioMaterial with the same name
my_custom_rm_bis = RadioMaterial(name='my_custom_mat', relative_permittivity=100000., bsdf=rm_itu_wood.bsdf)
    
# Assigning my_custom_rm_bis to an object would raise an error
# Two possible solutions to solve this
# First solution, modify the RadioMaterial of the scene
my_custom_rm.assign(my_custom_rm_bis)

# Check the permitivity 
print(f"The permitivity of my custom RadioMaterial is now {my_custom_rm.relative_permittivity}")
print(f"The two RadioMaterial are different python objects: {hex(id(my_custom_rm))} is not equal to {hex(id(my_custom_rm_bis))}")

# Need to reload the scene manually to update the rendering
scene.reload()

# Preview the scene
if colab_compat:
    scene.render(camera="scene-cam-0", num_samples=512)
    raise ExitCell
scene.preview()

As you can see, the .assign method for radio material does not automatically reload the scene.

Yet, the BSDF is correctly transfered from one RadioMaterial to another.

In [ ]:
my_custom_rm_bis.relative_permittivity = 10.
my_custom_rm_bis.bsdf.assign(rm_itu_metal.bsdf)

# Second solution, set the asset flag overwrite_scene_radio_materials to True when creating the AssetObject and change its RadioMaterial
# The overwrite action will be triggered when the asset is added to the scene
body_asset = AssetObject(name='asset_3', filename=sionna.rt.asset_object.body, radio_material=my_custom_rm_bis, overwrite_scene_radio_materials=True)

# Add the asset to the scene
# Since it has the same name as the other body asset, the one present in the scene will be replaced
scene.add(body_asset) # trigger a scene reload

# Check the permitivity 
print(f"The permitivity of my custom RadioMaterial is now {my_custom_rm.relative_permittivity}")

# Preview the scene
if colab_compat:
    scene.render(camera="scene-cam-0", num_samples=512)
    raise ExitCell
scene.preview()

## Annex Asset Features

In [ ]:
# Asset Look at

In [ ]:
# Asset velocity

## Asset Gradient Descent

In [ ]:
# Showcase gradient descent using a demongorgon and upside down pursuit

In [ ]:
# Remove single object from an asset or the scene

## Conclusion and Outlook